In [1]:
# Given the values of a single peakon on a coarse grid, this DNN predicts the value of the peakon at a particular location.
# Essentially, given the values shown by the @ and G symbols, and a location within the ... region, it will output its
# prediction for the value at that location.
# (In terms of the gen_one_data function below, then (t, x) is the location of the G symbol and (t2, x2) is the random
# location within the ... region.)
#
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @
#
#       .........
#       .       .
# @   @ . @   @ . @   @
#       .       .
#       .       .
#       .       .
# @   @ . G   @ . @   @
#       .       .
#       .........
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @

In [2]:
import tensorflow as tf

# https://github.com/patrick-kidger/tools
import tools

from base import ml

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [ ]:
for network_width in [400, 600, 800]:
    for network_depth in [2, 4, 6]:
        ### DNN hyperparameters
        hidden_units = [network_width] * network_depth
        logits = 1
        processor = ml.ScaleCenterAtPoint()
        activation=tf.nn.crelu

        ### Training hyperparameters #1
        drop_rate = 0.0
        drop_type = 'dropout'
        gradient_clip = 1.0  # May be set to None to turn off gradient clipping

        ### Logging/saving hyperparameters
        log_steps = 1000  # How many steps to print the current loss.
        model_dir_base = './saved_models/point_multi_peakon/'
        uuid = None
        model_dir = ml.model_dir_str(model_dir_base, hidden_units, logits,
                                     processor=processor, activation=activation, 
                                     uuid=uuid)

        dnn_factory = ml.DNNFactory(hidden_units, logits, drop_rate=drop_rate,
                                    drop_type=drop_type, model_dir=model_dir, 
                                    log_steps=log_steps, activation=activation, 
                                    gradient_clip=gradient_clip, batch_norm=False,
                                    processor=processor)

        max_steps = 0
        if TRAIN:
            ### Training hyperparameters #2
            batch_size = 50
            max_steps += 5000
            train_input_fn = ml.BatchData.from_func(gen_one_data=ml.gen_peakons_at_point, 
                                                    batch_size=batch_size)
            dnn = dnn_factory()
            dnn.train(input_fn=train_input_fn, max_steps=max_steps)

            batch_size = 500
            max_steps += 5000
            train_input_fn = ml.BatchData.from_func(gen_one_data=ml.gen_peakons_at_point, 
                                                    batch_size=batch_size)
            dnn = dnn_factory()
            dnn.train(input_fn=train_input_fn, max_steps=max_steps)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/point_single_peakon/400x2_1__ScaleCenterAtPoint_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7a29d2d278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./saved_models/point_s